# B_c_3_Test_Statistiques


In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import calendar
from references import colors_pal
from src.visualization import *

plt.style.use("fivethirtyeight")

pd.options.mode.chained_assignment = None

from patsy import dmatrices
import statsmodels.api as sm

path_to_interim_data = "../data/interim/"
demande_meteo_parquet = "demande_meteo.parquet"

df_import = pd.read_parquet(
    path=os.path.join(path_to_interim_data, demande_meteo_parquet),
    engine="pyarrow",
)
df = df_import["20190101":"20221231"]
df["DeltaTemp"] = abs(df["Temp"] - 18)
df["DeltaTempSigne"] = df["Temp"] - 18

df["idx_Mois"] = df.index.month
df["Mois"] = df["idx_Mois"].apply(lambda x: calendar.month_name[x].capitalize())

In [9]:
# Ref : https://www.statsmodels.org/stable/gettingstarted.html

df_stats = df.reset_index(drop=True).dropna()
df_stats = df_stats[["Mois", "MW", "Temp", "DeltaTemp"]]

y, X = dmatrices("MW ~ Mois + Temp + DeltaTemp", data=df_stats, return_type="dataframe")
mod = sm.OLS(y, X)  # Describe model
res = mod.fit()  # Fit model
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     MW   R-squared:                       0.843
Model:                            OLS   Adj. R-squared:                  0.843
Method:                 Least Squares   F-statistic:                 1.444e+04
Date:                Dim, 19 nov 2023   Prob (F-statistic):               0.00
Time:                        16:24:43   Log-Likelihood:            -3.1720e+05
No. Observations:               35018   AIC:                         6.344e+05
Df Residuals:                   35004   BIC:                         6.346e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept          1.407e+04     87.156    161.436      0.000    1.39e+04    1.42e+04
Mois[T.Avril]      1007.7423     63.585     15.849      0.000     883.114    1132.371
Mois[T.Décembre]   4704.1166     73.270     64.202      0.000    4560.505    4847.729
Mois[T.Février]    5433.7533     81.765     66.456      0.000    5273.491    5594.016
Mois[T.Janvier]    5703.2906     83.883     67.991      0.000    5538.876    5867.705
Mois[T.Juillet]    -245.9901     54.133     -4.544      0.000    -352.093    -139.887
Mois[T.Juin]       -235.7970     54.879     -4.297      0.000    -343.362    -128.232
Mois[T.Mai]        -381.1617     56.991     -6.688      0.000    -492.865    -269.458
Mois[T.Mars]       3494.2955     71.134     49.123      0.000    3354.870    3633.721
Mois[T.Novembre]   2338.4265     66.320     35.260      0.000    2208.438    2468.415
Mois[T.Octobre]     -64.6945     59.170     -1.093      0.274    -180.669      51.280
Mois[T.Septembre]  -251.0534     56.501     -4.443      0.000    -361.796    -140.310
Temp                102.8826      3.355     30.667      0.000      96.307     109.458
DeltaTemp           377.8253      3.673    102.862      0.000     370.626     385.025
==============================================================================
Omnibus:                      129.365   Durbin-Watson:                   0.116
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              119.811
Skew:                          -0.111   Prob(JB):                     9.63e-27
Kurtosis:                       2.819   Cond. No.                         281.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
res.params

Intercept            14070.141645
Mois[T.Avril]         1007.742284
Mois[T.Décembre]      4704.116617
Mois[T.Février]       5433.753320
Mois[T.Janvier]       5703.290616
Mois[T.Juillet]       -245.990127
Mois[T.Juin]          -235.797013
Mois[T.Mai]           -381.161672
Mois[T.Mars]          3494.295519
Mois[T.Novembre]      2338.426542
Mois[T.Octobre]        -64.694482
Mois[T.Septembre]     -251.053399
Temp                   102.882623
DeltaTemp              377.825335
dtype: float64

In [11]:
# https://www.statsmodels.org/stable/gettingstarted.html#diagnostics-and-specification-tests
# Rainbow test for linearity (the null hypothesis is that the relationship is properly modelled as linear)

sm.stats.linear_rainbow(res)

(1.092009849879139, 2.9205866375105065e-09)